In [1]:
import numpy as np # https://numpy.org/
import pandas as pd # https://pandas.pydata.org/
import seaborn as sns # https://seaborn.pydata.org/
import matplotlib.pyplot as plt # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
%matplotlib inline

In [2]:
df = pd.read_csv('precio_prep_reg.csv', header = 0) # Leer el archivo generado en el notebook de preproceso

In [3]:
X_multiple = df.drop('precio', axis = 1) # Crear un nuevo dataframe sin la columna precio (axis=1 -> borra columna)

In [4]:
y_multiple = df['precio']

In [5]:
from sklearn.linear_model import LinearRegression # Importar el modulo y la función necesaria para el modelo de regresión lineal


# División de los datos en set de entrenamiento y validación

In [6]:
from sklearn.model_selection import train_test_split # Importar modulo y función necesaria para hacer una división
# aleatoria de los datos

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_multiple, y_multiple, test_size = 0.2, random_state = 123) # Obtener los cuatro
# conjuntos de datos necesarios para el modelo

In [8]:
X_train.shape, X_test.shape, y_test.shape, y_train.shape # Revisar las dimensiones de las matrices de los dataframes generados

((404, 16), (102, 16), (102,), (404,))

In [9]:
linear_model_s = LinearRegression()

In [10]:
linear_model_s.fit(X_train, y_train)

LinearRegression()

In [11]:
y_train_s = linear_model_s.predict(X_train) # Realizar la predicción en el conjunto de entrenamiento

In [12]:
y_test_s = linear_model_s.predict(X_test) # Realizar la predicción en el conjunto de validación

In [13]:
y_test_s # Ver las predicciones desarrolladas por el modelo para el conjunto de validación X_test

array([21.89180282, 28.84871612, 37.88418257, 17.99045587, 30.82693682,
       35.38575116, 27.03504994, 11.88392855, 12.27713853, 30.27525997,
       28.44895244, 19.47018107, 16.04044274, 31.97928247, 16.13008315,
       20.75694421, 21.77160421, 35.90669821, 18.37906422, 13.5323649 ,
       13.90293317, 24.58737059, 33.13662615, 38.77680983, 29.72274023,
       20.89157897, 16.141351  , 19.80877264, 25.09614865, 21.73136166,
       27.00224361, 33.40274297, 18.99731452, 21.31226678, 23.81879853,
       33.6061337 , 27.62278577, 15.94452147, 15.01268021, 37.18176829,
       31.1979934 , 20.31047201, 19.00915542, 40.82508937,  6.38615163,
       29.73649579, 20.89938075, 27.04510078, 16.28504157, 26.10585375,
       32.67333847, 24.41206431, 20.47220667,  9.70510879, 25.45090166,
       17.95944975, 10.31318479,  0.16599375, 29.80740332,  9.78099981,
       21.06926222, 21.73020023, 22.13621234, 17.69717614, 21.47723338,
       24.01432364, 24.09636865, 23.97487197, 23.79645885, 31.28

In [14]:
X_test # Ver el conjunto de validación

,tasa_crimen,area_resid,cal_aire,num_hab,edad,prof_mil,p_pobreza,n_cam_hos,n_hab_hotel,lluvia_ano,parque,dist_prom,aeropuerto_SI,cuerpo_agua_Lago,cuerpo_agua_Lago y Rio,cuerpo_agua_Rio
410,3.953852,48.10,0.5970,5.757,100.0,19.8,10.11,9.600,11.1200,52,0.061616,1.4125,0,0,0,1
85,0.055766,34.49,0.4490,6.630,56.1,21.5,6.53,9.732,12.2128,56,0.052066,4.4400,0,0,0,1
280,0.035155,33.33,0.4429,7.820,64.5,25.1,3.76,7.108,10.3632,50,0.041951,4.6950,0,0,0,0
422,2.568650,48.10,0.6140,5.648,87.6,19.8,14.10,8.916,11.1664,36,0.058066,1.9525,0,0,0,0
199,0.031014,31.47,0.4030,6.975,15.3,23.0,4.56,7.998,13.2792,24,0.041284,7.6525,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,0.365878,36.20,0.5040,6.552,21.4,22.6,3.76,7.330,15.2520,47,0.044352,3.3725,1,0,0,0
159,0.885840,49.58,0.8710,6.510,100.0,25.3,7.39,7.166,12.1864,36,0.076489,1.7675,1,1,0,0
196,0.039326,31.52,0.4040,7.287,34.1,27.4,4.08,7.266,15.2664,20,0.036636,7.3100,1,0,1,0
345,0.030655,34.39,0.4420,6.014,48.5,21.2,10.53,7.150,13.1400,26,0.050677,8.0125,1,1,0,0


In [15]:
y_test_s_df = pd.DataFrame(y_test_s, columns = ['Prediccion']) # Convertir el array vector de predicciones a dataframe

In [16]:
y_test_s_df

,Prediccion
0,21.891803
1,28.848716
2,37.884183
3,17.990456
4,30.826937
...,...
97,32.233031
98,28.213022
99,35.645679
100,19.638558


In [17]:
X_test_reindex = X_test.reset_index(drop = True, inplace = False) # Reindexar X_texst para poder hacer el join con y_test_s_df

In [18]:
test_x_con_pred = X_test_reindex.join(y_test_s_df) # Realizar la fusión de los dos datafram para poder ver el las variables predictoras
# y la variable predica en un solo dataframe

In [19]:
test_x_con_pred

,tasa_crimen,area_resid,cal_aire,num_hab,edad,prof_mil,p_pobreza,n_cam_hos,n_hab_hotel,lluvia_ano,parque,dist_prom,aeropuerto_SI,cuerpo_agua_Lago,cuerpo_agua_Lago y Rio,cuerpo_agua_Rio,Prediccion
0,3.953852,48.10,0.5970,5.757,100.0,19.8,10.11,9.600,11.1200,52,0.061616,1.4125,0,0,0,1,21.891803
1,0.055766,34.49,0.4490,6.630,56.1,21.5,6.53,9.732,12.2128,56,0.052066,4.4400,0,0,0,1,28.848716
2,0.035155,33.33,0.4429,7.820,64.5,25.1,3.76,7.108,10.3632,50,0.041951,4.6950,0,0,0,0,37.884183
3,2.568650,48.10,0.6140,5.648,87.6,19.8,14.10,8.916,11.1664,36,0.058066,1.9525,0,0,0,0,17.990456
4,0.031014,31.47,0.4030,6.975,15.3,23.0,4.56,7.998,13.2792,24,0.041284,7.6525,1,0,0,0,30.826937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,0.365878,36.20,0.5040,6.552,21.4,22.6,3.76,7.330,15.2520,47,0.044352,3.3725,1,0,0,0,32.233031
98,0.885840,49.58,0.8710,6.510,100.0,25.3,7.39,7.166,12.1864,36,0.076489,1.7675,1,1,0,0,28.213022
99,0.039326,31.52,0.4040,7.287,34.1,27.4,4.08,7.266,15.2664,20,0.036636,7.3100,1,0,1,0,35.645679
100,0.030655,34.39,0.4420,6.014,48.5,21.2,10.53,7.150,13.1400,26,0.050677,8.0125,1,1,0,0,19.638558


In [20]:
from sklearn.metrics import r2_score # Importar la función que realiza el calculo del coeficiente de determinación
# con el fin de analizar la bondad de ajuste del modelo

In [21]:
r2_score?

Signature:
r2_score(
    y_true,
    y_pred,
    *,
    sample_weight=None,
    multioutput='uniform_average',
)
Docstring:
R^2 (coefficient of determination) regression score function.

Best possible score is 1.0 and it can be negative (because the
model can be arbitrarily worse). A constant model that always
predicts the expected value of y, disregarding the input features,
would get a R^2 score of 0.0.

Read more in the :ref:`User Guide <r2_score>`.

Parameters
----------
y_true : array-like of shape (n_samples,) or (n_samples, n_outputs)
    Ground truth (correct) target values.

y_pred : array-like of shape (n_samples,) or (n_samples, n_outputs)
    Estimated target values.

sample_weight : array-like of shape (n_samples,), default=None
    Sample weights.

multioutput : {'raw_values', 'uniform_average', 'variance_weighted'},             array-like of shape (n_outputs,) or None, default='uniform_average'

    Defines aggregating of multiple output scores.
    Array-like value defi

In [22]:
r2_score(y_train, y_train_s) # Ver la bondad de ajuste en los datos de entrenamiento

0.747689691306311

In [23]:
r2_score(y_test, y_test_s) # Ver la bondad de ajuste en los datos de validación

0.5957731490588167

## Validación cruzada (KFold)
![Validación cruzada](5-Fold-Cross-Validation.jpg)

In [24]:
from sklearn.model_selection import KFold # Importar el modulo necesario para desarrollar la validación cruzada
from sklearn.model_selection import cross_val_score

In [25]:
kf = KFold(n_splits = 10) # Escoger las porciones en que vamos a dividir el dataset de entrenamiento para hacer las particiones y las pruebas

In [26]:
score = linear_model_s.score(X_train, y_train) # Hallar el valor de r2 con la función score

In [27]:
score # Observar que es congruente con la respuesta obtenida por el metodo de división en entrenamiento y validación

0.747689691306311

In [28]:
score_cv_train = cross_val_score(linear_model_s, X_train, y_train, cv = kf) # Desarrollar la validación cruzada con el conjunto de datos de entrenamiento

In [29]:
score_cv_train

array([0.56409073, 0.71317068, 0.82670501, 0.82307103, 0.64894639,
       0.84640757, 0.57229203, 0.76101139, 0.4906286 , 0.70743903])

In [30]:
print("Media de las validaciones set de entrenamiento: ", score_cv_train.mean())

Media de las validaciones set de entrenamiento:  0.6953762471062404


Observar que con la validación cruzada se obtiene un valor de $r^2$ menor al obtenido por la validación simple o particionada.